### If there is a set of 10 positive numbers, are there two subsets that sum of the elements in both sets is equal?

#### objective function
$$minimize~~ (\sum_{i=1}^{5} n_{i}s_{i})^{2}$$

In [75]:
from pyqubo import Array
s=Array.create('s',shape=4,vartype='SPIN')   
H=sum(n*s for s,n in zip(s,range(1,5)))**2
model=H.compile()
qubo,offset=model.to_qubo()

In [76]:
print(H)

(((Spin(s[0])*Num(1))+(Spin(s[1])*Num(2))+(Spin(s[2])*Num(3))+(Spin(s[3])*Num(4)))*((Spin(s[0])*Num(1))+(Spin(s[1])*Num(2))+(Spin(s[2])*Num(3))+(Spin(s[3])*Num(4))))


In [77]:
print(model)

Model(CompiledQubo({('s[0]', 's[0]'): -36.0,
 ('s[0]', 's[1]'): 16.0,
 ('s[0]', 's[2]'): 24.0,
 ('s[0]', 's[3]'): 32.0,
 ('s[1]', 's[1]'): -64.0,
 ('s[1]', 's[2]'): 48.0,
 ('s[1]', 's[3]'): 64.0,
 ('s[2]', 's[2]'): -84.0,
 ('s[2]', 's[3]'): 96.0,
 ('s[3]', 's[3]'): -96.0}, offset=100.0), structure={'s[0]': ('s', 0), 's[1]': ('s', 1), 's[2]': ('s', 2), 's[3]': ('s', 3)})


In [78]:
bqm=model.to_dimod_bqm()

In [79]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import neal
sampler=EmbeddingComposite(DWaveSampler())
sampler=neal.SimulatedAnnealingSampler()
response=sampler.sample(bqm)

In [80]:
for sample in response.samples():
     print(sample)

{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1}
{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1}
{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1}
{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0}
{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0}
{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0}
{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1}
{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1}
{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0}
{'s[0]': 0, 's[1]': 1, 's[2]': 0, 's[3]': 1}


In [81]:
import pandas as pd
from collections import defaultdict    
result=defaultdict(list)
for r in response.data(['sample', 'energy']):
    result[r.energy].append(r.sample)

res=pd.DataFrame()
min_energy=min(result.keys())
for i in list(result[min_energy]):
    temp=pd.DataFrame.from_dict(i,'index').transpose()
    res=res.append(temp)
res=res.drop_duplicates()
print(res.iloc[0])

s[0]    1
s[1]    0
s[2]    0
s[3]    1
Name: 0, dtype: int64


In [82]:
df2=res.iloc[0].transpose().reset_index().reset_index()
df2['level_0']=df2['level_0']+1
ans={'group1':{'list':list(df2[df2[0]==0]['level_0']),'sum':sum(list(df2[df2[0]==0]['level_0']))},
    'group2:':{'list':list(df2[df2[0]==1]['level_0']),'sum':sum(list(df2[df2[0]==1]['level_0']))}}
print(pd.DataFrame.from_dict(ans))

      group1 group2:
list  [2, 3]  [1, 4]
sum        5       5


### Show more answers

In [83]:
response=sampler.sample(bqm,num_reads=100)

In [84]:
result=defaultdict(int)
for datum in response.data(['sample', 'energy', 'num_occurrences']):   
    #print(str(datum.sample), "Energy: ", datum.energy, "Occurrences: ", datum.num_occurrences)
    result[str(datum.sample)+' '+str(datum.energy)]+=1
print(pd.DataFrame.from_dict(result,'index').rename(columns={0:'num_occurrences'}))

                                                  num_occurrences
{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0} 0.0               50
{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1} 0.0               49
{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 1} 4.0                1


### Add constraint to balance the size of two sets

In [105]:
from pyqubo import Array
s=Array.create('s',shape=4,vartype='SPIN')   
M=1 #constraint parameter
H=sum(n*s for s,n in zip(s,range(1,5)))**2+M*Constraint(sum(s)**2,label='(sum s)^2= 0')
model=H.compile()
qubo,offset=model.to_qubo()

In [106]:
print(H)

((((Spin(s[0])*Num(1))+(Spin(s[1])*Num(2))+(Spin(s[2])*Num(3))+(Spin(s[3])*Num(4)))*((Spin(s[0])*Num(1))+(Spin(s[1])*Num(2))+(Spin(s[2])*Num(3))+(Spin(s[3])*Num(4))))+(Const((sum s)^2= 0, ((Spin(s[0])+Spin(s[1])+Spin(s[2])+Spin(s[3]))*(Spin(s[0])+Spin(s[1])+Spin(s[2])+Spin(s[3]))))*Num(1)))


In [107]:
model=H.compile()
qubo,offset=model.to_qubo()
bqm=model.to_dimod_bqm()

In [128]:
sampler=EmbeddingComposite(DWaveSampler(endpoint='https://cloud.dwavesys.com/sapi', token='DEV-6868dc20cabd03c8bcdd24a96b33799beaa79865', solver='DW_2000Q_2_1'))
sampler=neal.SimulatedAnnealingSampler()
response=sampler.sample(bqm,num_reads=100)

In [129]:
result=defaultdict(int)
for datum in response.data(['sample', 'energy', 'num_occurrences']):   
    result[str(datum.sample)+' '+str(datum.energy)]+=1
print(pd.DataFrame.from_dict(result,'index').rename(columns={0:'num_occurrences'}))

                                                  num_occurrences
{'s[0]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 1} 0.0               44
{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0} 0.0               52
{'s[0]': 0, 's[1]': 1, 's[2]': 0, 's[3]': 1} 4.0                3
{'s[0]': 1, 's[1]': 0, 's[2]': 1, 's[3]': 0} 4.0                1


In [130]:
import pandas as pd
from collections import defaultdict    
result=defaultdict(list)
for r in response.data(['sample', 'energy']):
    result[r.energy].append(r.sample)

res=pd.DataFrame()
min_energy=min(result.keys())
for i in list(result[min_energy]):
    temp=pd.DataFrame.from_dict(i,'index').transpose()
    res=res.append(temp)
res=res.drop_duplicates()
print(res.iloc[0])

s[0]    1
s[1]    0
s[2]    0
s[3]    1
Name: 0, dtype: int64


In [131]:
df2=res.iloc[0].transpose().reset_index().reset_index()
df2['level_0']=df2['level_0']+1
ans={'group1':{'list':list(df2[df2[0]==0]['level_0']),'sum':sum(list(df2[df2[0]==0]['level_0']))},
    'group2:':{'list':list(df2[df2[0]==1]['level_0']),'sum':sum(list(df2[df2[0]==1]['level_0']))}}
print(pd.DataFrame.from_dict(ans))

      group1 group2:
list  [2, 3]  [1, 4]
sum        5       5
